In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-05"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
2814,2024-06-05,Lituânia Lkl,12:50,Zalgiris Kaunas,Rytas,1.34,3.14,170.5,1.87,1.87,0.0,0.00,0.00,K6pE0D7l,0.746269,0.318471,0.534759,0.534759,0.064740,106.388,10.749080,0.101037,2.4,1.341641,0.559017,96.80,1.166,0.106677,0.091490,57.0,128.456,22.882734,0.178137,1.8,1.643168,0.912871,126.96,1.384,0.062290,0.045007,30.0,88,92,1.10,1.38,99.884,125.834,0.568211,0.000000,NaN,0.32,0.064,5.312500,0.906011,1.0,0.093989,-1.37,-0.274,-7.810219,0.684290,0.6,-0.084290
2815,2024-06-05,México Cibacopa,00:15,Rayos de Hermosillo,Halcones de Obregon,1.34,3.06,185.5,1.86,1.80,0.0,0.00,0.00,GlCZfud4,0.746269,0.326797,0.537634,0.555556,0.073066,135.272,22.948472,0.169647,1.2,1.643168,1.369306,147.90,1.496,0.186628,0.124751,-8.0,250.536,54.764839,0.218591,1.8,1.643168,0.912871,207.05,2.680,0.611637,0.228223,14.0,87,101,1.70,2.05,116.000,264.668,0.552829,0.023184,NaN,-0.91,-0.182,-1.868132,0.702102,0.6,-0.102102,-2.07,-0.414,-4.975845,0.460421,0.2,-0.260421
2816,2024-06-05,Nova Zelândia Nbl,04:30,Wellington Saints,Manawatu Jets,1.02,12.50,191.5,1.85,1.85,0.0,0.00,0.00,hEa2mthC,0.980392,0.080000,0.540541,0.540541,0.060392,157.342,66.883202,0.425082,1.2,1.643168,1.369306,265.32,1.674,0.691614,0.413151,-12.0,577.072,363.153682,0.629304,0.0,0.000000,NaN,230.56,7.690,5.151864,0.669943,-114.0,99,88,2.68,2.62,0.000,543.768,1.200826,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,-5.00,-1.000,-11.500000,0.000000,0.0,0.000000
2817,2024-06-05,Polônia Liga De Basquete,15:00,Szczecin,Trefl Sopot,1.49,2.54,163.5,1.87,1.79,0.0,0.00,0.00,vD8gc5J0,0.671141,0.393701,0.534759,0.558659,0.064842,152.498,32.184789,0.211051,2.4,1.341641,0.559017,182.25,1.780,0.442380,0.248528,43.0,120.934,11.145357,0.092161,1.8,1.643168,0.912871,135.24,1.532,0.128919,0.084151,10.0,81,84,2.25,1.61,130.192,141.676,0.368468,0.030912,NaN,1.01,0.202,2.425743,0.741278,0.8,0.058722,-0.10,-0.020,-77.000000,0.621707,0.7,0.078293
2818,2024-06-05,Portugal Lpb,15:00,FC Porto,Benfica,2.20,1.62,158.5,1.85,1.85,2.5,1.90,1.56,E9w8IZgS,0.454545,0.617284,0.540541,0.540541,0.071829,90.126,12.034055,0.133525,2.4,1.341641,0.559017,108.36,1.112,0.091488,0.082273,52.0,94.804,11.491263,0.121211,3.0,0.000000,0.000000,86.24,1.046,0.043932,0.042000,152.0,86,77,1.26,1.12,92.030,93.526,0.214724,0.000000,0.138969,-0.51,-0.102,-11.764706,0.932410,0.9,-0.032410,-0.68,-0.136,-4.558824,0.000000,0.0,0.000000
2819,2024-06-05,Venezuela Superliga,20:00,Cocodrilos,Heroes de Falcon,1.12,5.89,157.5,1.83,1.83,0.0,0.00,0.00,YHjJrgWa,0.892857,0.169779,0.546448,0.546448,0.062636,133.602,33.494706,0.250705,3.0,0.000000,0.000000,154.66,1.488,0.372653,0.250439,78.0,129.214,28.117082,0.217601,1.8,1.643168,0.912871,100.64,1.568,0.351383,0.224096,28.0,74,68,2.09,1.48,0.000,0.000,0.962311,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
2820,2024-06-05,Sérvia Superliga,15:00,Partizan,Crvena zvezda,1.31,3.25,167.5,1.87,1.83,0.0,0.00,0.00,Q1WetkLt,0.763359,0.307692,0.534759,0.546448,0.071051,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,88,89,2.80,1.40,0.000,0.000,0.601661,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
2821,2024-06-05,Porto Ric

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
2835,21:30,Bolívia Libobasquet,Carl A-Z,La Salle Tarija,2.31,Back Home
2841,21:00,Porto Rico Bsn,Osos de Manati,Cangrejeros,1.71,Back Home
